Data Scraping Notebook

In [191]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

# Import the API key
from config import geoapify_key
from config import census_key


# Launch the browser
browser = Browser('chrome')

Create a "dictiornary" lists listing all states - Createing the list will help to make sure all states are listed and no state data will be lot in dataframe merges that will follow. 

In [192]:
# Set the URL to visit to a variable
url_states = "https://developers.google.com/public-data/docs/canonical/states_csv"

# Send the browser to the URL
browser.visit(url_states)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [193]:
#find all divs that contain state marker
table_state = soup.find('div', class_='devsite-table-wrapper')

In [194]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state = {}

# Find the table element
state_list = []


# Check if the table is found
if table_state:
    # Find all rows in the table body
    rows = table_state.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            lat = cells[1].text.strip()
            lon = cells[2].text.strip()
            name = cells[3].text.strip()
            
            # Create a dictionary for the current state
            state = {'state_abbr': state, 'state': name}
            
            # Append the state data dictionary to the list
            state_list.append(state)

# Print the extracted data
#for state in state_list:
    #print(state)

In [195]:
#load scraped state data into dataframe
state_df = pd.DataFrame(state_list)
#save dataframe to csv
state_df.to_csv('Resources/state_list.csv', index=False)

1. Which states require payment of state taxes on gambling winnings?

 A table showing the latest information regarding which states tax gambling winnings. Keep in mind that each state that requires state tax on gambling income has its own tax rate and guidelines. In some cases, gambling winnings are subject to state tax except for lottery winnings.

In [196]:
# Set the URL to visit to a variable
url = "https://www.playusa.com/taxes/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [197]:
#find all divs that contain state marker
table = soup.find('table', class_='tablepress-responsive')

In [198]:
#Extract state-specific tax data from the table and build a dictionary of the data. 
state_tax = {}

# Find the table element
state_tax_list = []

# Find the table element
#table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if table:
    # Find all rows in the table body
    rows = table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            state = cells[0].text.strip()
            tax_status = cells[1].text.strip()
            
            # Create a dictionary for the current state
            state_data = {'state': state, 'tax_status': tax_status}
            
            # Append the state data dictionary to the list
            state_tax_list.append(state_data)

# Print the extracted data
for state_data in state_tax_list:
    print(state_data)

{'state': 'Alabama', 'tax_status': 'Yes'}
{'state': 'Alaska', 'tax_status': 'No'}
{'state': 'Arizona', 'tax_status': 'Yes'}
{'state': 'Arkansas', 'tax_status': 'Yes'}
{'state': 'California', 'tax_status': 'No'}
{'state': 'Colorado', 'tax_status': 'Yes'}
{'state': 'Connecticut', 'tax_status': 'Yes'}
{'state': 'Delaware', 'tax_status': 'Yes'}
{'state': 'D.C.', 'tax_status': 'Yes'}
{'state': 'Florida', 'tax_status': 'No'}
{'state': 'Georgia', 'tax_status': 'Yes'}
{'state': 'Hawaii', 'tax_status': 'No'}
{'state': 'Idaho', 'tax_status': 'Yes'}
{'state': 'Illinois', 'tax_status': 'Yes'}
{'state': 'Indiana', 'tax_status': 'Yes'}
{'state': 'Iowa', 'tax_status': 'Yes'}
{'state': 'Kansas', 'tax_status': 'Yes'}
{'state': 'Kentucky', 'tax_status': 'Yes'}
{'state': 'Louisiana', 'tax_status': 'Yes'}
{'state': 'Maine', 'tax_status': 'Yes'}
{'state': 'Maryland', 'tax_status': 'Yes'}
{'state': 'Massachusetts', 'tax_status': 'Yes'}
{'state': 'Michigan', 'tax_status': 'Yes'}
{'state': 'Minnesota', 'tax_s

In [199]:
#create a dataframe from the state_data dictionary of lists and display the first 5 rows
state_tax_df = pd.DataFrame(state_tax_list)
state_tax_df.head()

,state,tax_status
0,Alabama,Yes
1,Alaska,No
2,Arizona,Yes
3,Arkansas,Yes
4,California,No


In [200]:
#replace D.C. to District of Columbia in the state column
state_tax_df['state'] = state_tax_df['state'].replace('D.C.', 'District of Columbia')

Combining the tax information with the full list of states to ensure all states are listed for merges that follow. 

In [201]:
#merge state_df and state_tax_df
state_tax_df = state_df.merge(state_tax_df, on='state', how='left')

In [202]:
state_tax_df.to_csv('Resources/state_tax_df.csv', index=False)

2. Scrape the running total for sports betting handle and revenue reported to date across all active markets:

In [203]:
# Set the URL to visit to a variable
url = "https://www.legalsportsreport.com/sports-betting/revenue/"

# Send the browser to the URL
browser.visit(url)

# Save the HTML from the browser
html = browser.html

# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, 'html.parser')

In [204]:
rev_table = soup.find('table', class_='tablepress-id-Rev_Markets')

In [205]:
# Create an empty list to store market data
market_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if rev_table:
    # Find all rows in the table body
    rows = rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            market = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            market_data = {'state': market, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            market_data_list.append(market_data)

# Print the extracted data
for market_data in market_data_list:
    print(market_data)

{'state': 'New Jersey', 'handle': '$39,461,272,422', 'revenue': '$2,871,419,711', 'hold': '7.3%', 'taxes': '$397,552,569'}
{'state': 'Nevada', 'handle': '$33,520,083,578', 'revenue': '$1,885,621,000', 'hold': '5.6%', 'taxes': '$127,279,418'}
{'state': 'New York', 'handle': '$25,545,018,522', 'revenue': '$2,231,885,589', 'hold': '8.7%', 'taxes': '$1,116,577,865'}
{'state': 'Illinois', 'handle': '$24,059,865,704', 'revenue': '$1,962,146,676', 'hold': '8.2%', 'taxes': '$312,219,908'}
{'state': 'Pennsylvania', 'handle': '$22,766,881,865', 'revenue': '$1,856,291,582', 'hold': '8.2%', 'taxes': '$462,263,411'}
{'state': 'Colorado', 'handle': '$12,795,525,493', 'revenue': '$857,637,396', 'hold': '6.7%', 'taxes': '$46,491,000'}
{'state': 'Indiana', 'handle': '$12,751,685,026', 'revenue': '$1,085,683,295', 'hold': '8.5%', 'taxes': '$103,236,082'}
{'state': 'Michigan', 'handle': '$11,067,854,802', 'revenue': '$950,198,739', 'hold': '8.6%', 'taxes': '$48,127,549'}
{'state': 'Arizona', 'handle': '$

In [206]:
market_rev_df = pd.DataFrame(market_data_list)
market_rev_df.head(51)

,state,handle,revenue,hold,taxes
0,New Jersey,"$39,461,272,422","$2,871,419,711",7.3%,"$397,552,569"
1,Nevada,"$33,520,083,578","$1,885,621,000",5.6%,"$127,279,418"
2,New York,"$25,545,018,522","$2,231,885,589",8.7%,"$1,116,577,865"
3,Illinois,"$24,059,865,704","$1,962,146,676",8.2%,"$312,219,908"
4,Pennsylvania,"$22,766,881,865","$1,856,291,582",8.2%,"$462,263,411"
5,Colorado,"$12,795,525,493","$857,637,396",6.7%,"$46,491,000"
6,Indiana,"$12,751,685,026","$1,085,683,295",8.5%,"$103,236,082"
7,Michigan,"$11,067,854,802","$950,198,739",8.6%,"$48,127,549"
8,Arizona,"$11,005,985,362","$907,366,992",8.2%,"$51,509,590"
9,Virginia,"$10,748,305,916","$1,037,483,246",9.7%,"$106,594,915"


In [207]:
#replace "?"" in market_rev_df for handle, revenue, hold, and taxes with 0
market_rev_df = market_rev_df.replace({'?': 0})

Combining the tax information with the full list of states to ensure all states are listed for merges that follow. 

In [208]:
#merge state_df and market_rev_df by state
state_rev_df = state_df.merge(market_rev_df, on='state', how='left')
#state_rev_df.head()
state_rev_df.head(51)

,state_abbr,state,handle,revenue,hold,taxes
0,AK,Alaska,NaN,NaN,NaN,NaN
1,AL,Alabama,NaN,NaN,NaN,NaN
2,AR,Arkansas,"$486,217,898","$49,119,871",10.1%,"$7,134,908"
3,AZ,Arizona,"$11,005,985,362","$907,366,992",8.2%,"$51,509,590"
4,CA,California,NaN,NaN,NaN,NaN
5,CO,Colorado,"$12,795,525,493","$857,637,396",6.7%,"$46,491,000"
6,CT,Connecticut,"$2,739,546,897","$276,599,033",10.1%,"$27,911,334"
7,DC,District of Columbia,"$598,243,001","$73,848,263",12.3%,"$18,623,076"
8,DE,Delaware,"$567,831,224","$112,283,049",19.8%,"$66,008,234"
9,FL,Florida,NaN,NaN,NaN,NaN


In [209]:
#replace NaN with 0
state_rev_df = state_rev_df.fillna(0)

Adding the sports league detail to the revenue by state df.

In [210]:
#add sport league info to the market revenue data frame - data added from a csv file copied from WIKI https://en.wikipedia.org/wiki/Major_professional_sports_teams_of_the_United_States_and_Canada#:~:text=Teams%20%20%20%20Team%20%20%20,%20%20MLB%20%2018%20more%20rows%20
#file previously copied directly and saved in the csv format
#read the csv file
us_major_teams_df = pd.read_csv('Resources/us_major_teams.csv')
us_major_teams_df.head()


,Team,Venue,City,State/Province,League
0,Arizona Diamondbacks,Chase Field,Phoenix,Arizona,MLB
1,Phoenix Suns,Footprint Center,Phoenix,Arizona,NBA
2,Arizona Cardinals,State Farm Stadium,Glendale,Arizona,NFL
3,Arizona Coyotes,Mullett Arena,Tempe,Arizona,NHL
4,Anaheim Ducks,Honda Center,Anaheim,California,NHL


In [211]:
# Create a summary DataFrame to count unique teams and leagues per state
summary_data = []

# Get unique states
unique_states = us_major_teams_df['State/Province'].unique()

for state in unique_states:
    state_teams = us_major_teams_df[us_major_teams_df['State/Province'] == state]
    total_teams = state_teams['Team'].nunique()
    
    state_summary = {'state': state, 'Teams Count': total_teams}
    
    # Count teams for each league
    leagues = ['MLB', 'NBA', 'NFL', 'NHL', 'MLS', 'NCAA']
    for league in leagues:
        league_count = state_teams[state_teams['League'] == league]['Team'].nunique()
        state_summary[league] = league_count
    
    summary_data.append(state_summary)

# Create the summary DataFrame
summary_df = pd.DataFrame(summary_data)

# Display the summary DataFrame
print(summary_df)



                   state  Teams Count  MLB  NBA  NFL  NHL  MLS  NCAA
0                Arizona            4    1    1    1    1    0     0
1             California           19    5    4    3    3    4     0
2               Colorado            5    1    1    1    1    1     0
3   District of Columbia            4    1    1    0    1    1     0
4                Florida           11    2    2    3    2    2     0
5                Georgia            4    1    1    1    0    1     0
6               Illinois            6    2    1    1    1    1     0
7                Indiana            2    0    1    1    0    0     0
8                 Kansas            1    0    0    0    0    1     0
9              Louisiana            2    0    1    1    0    0     0
10              Maryland            3    1    0    2    0    0     0
11         Massachusetts            5    1    1    1    1    1     0
12              Michigan            4    1    1    1    1    0     0
13             Minnesota          

In [212]:
#merge market_rev_df with summary_df with by state with join on state 
merged_state_rev_league_df = pd.merge(market_rev_df, summary_df, on='state', how='left')

# Display the merged DataFrame
merged_state_rev_league_df.head(51)


,state,handle,revenue,hold,taxes,Teams Count,MLB,NBA,NFL,NHL,MLS,NCAA
0,New Jersey,"$39,461,272,422","$2,871,419,711",7.3%,"$397,552,569",4.0,0.0,0.0,2.0,1.0,1.0,0.0
1,Nevada,"$33,520,083,578","$1,885,621,000",5.6%,"$127,279,418",2.0,0.0,0.0,1.0,1.0,0.0,0.0
2,New York,"$25,545,018,522","$2,231,885,589",8.7%,"$1,116,577,865",9.0,2.0,2.0,1.0,3.0,1.0,0.0
3,Illinois,"$24,059,865,704","$1,962,146,676",8.2%,"$312,219,908",6.0,2.0,1.0,1.0,1.0,1.0,0.0
4,Pennsylvania,"$22,766,881,865","$1,856,291,582",8.2%,"$462,263,411",8.0,2.0,1.0,2.0,2.0,1.0,0.0
5,Colorado,"$12,795,525,493","$857,637,396",6.7%,"$46,491,000",5.0,1.0,1.0,1.0,1.0,1.0,0.0
6,Indiana,"$12,751,685,026","$1,085,683,295",8.5%,"$103,236,082",2.0,0.0,1.0,1.0,0.0,0.0,0.0
7,Michigan,"$11,067,854,802","$950,198,739",8.6%,"$48,127,549",4.0,1.0,1.0,1.0,1.0,0.0,0.0
8,Arizona,"$11,005,985,362","$907,366,992",8.2%,"$51,509,590",4.0,1.0,1.0,1.0,1.0,0.0,0.0
9,Virginia,"$10,748,305,916","$1,037,483,246",9.7%,"$106,594,915",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
#show datatype for each column 
merged_state_rev_league_df.dtypes

state           object
handle          object
revenue         object
hold            object
taxes           object
Teams Count    float64
MLB            float64
NBA            float64
NFL            float64
NHL            float64
MLS            float64
NCAA           float64
dtype: object

In [215]:
#Datframe cleanup before loading into csv:
merged_state_rev_league_df = merged_state_rev_league_df.fillna(0)
merged_state_rev_league_df['handle'] = merged_state_rev_league_df['handle'].str.replace('[\$,]', '', regex=True).astype(float)
merged_state_rev_league_df['revenue'] = merged_state_rev_league_df['revenue'].str.replace('[\$,]', '', regex=True).astype(float)
merged_state_rev_league_df['hold'] = merged_state_rev_league_df['hold'].apply(lambda x: float(str(x).replace('%', '')) / 100)
merged_state_rev_league_df['taxes'] = merged_state_rev_league_df['taxes'].str.replace('[\$,]', '', regex=True).astype(float)



AttributeError: Can only use .str accessor with string values!

In [32]:
market_rev_df.to_csv('Resources/market_rev_df.csv', index=False)

3. US sports betting revenue by month

In [33]:
month_rev_table = soup.find('table', class_='tablepress-id-Rev_Months')

In [34]:
# Create an empty list to store market data
month_data_list = []

# Find the table element
#rev_table = soup.find('table', {'class': 'tablepress'})

# Check if the table is found
if month_rev_table:
    # Find all rows in the table body
    rows = month_rev_table.select('tbody tr')

    # Loop through the rows
    for row in rows:
        # Find all cells in the row
        cells = row.select('td')
        
        # Check if there are cells in the row
        if cells:
            # Extract data from the cells
            month = cells[0].text.strip()
            handle = cells[1].text.strip()
            revenue = cells[2].text.strip()
            hold = cells[3].text.strip()
            taxes = cells[4].text.strip()
            
            # Create a dictionary for the current market
            timeline_data = {'month': month, 'handle': handle, 'revenue': revenue, 'hold': hold, 'taxes': taxes}
            
            # Append the market data dictionary to the list
            month_data_list.append(timeline_data)

# Print the extracted data
for timeline_data in month_data_list:
    print(timeline_data)

{'month': 'June 2018', 'handle': '$309,695,659', 'revenue': '$24,630,915', 'hold': '8.0%', 'taxes': '$2,221,650'}
{'month': 'July 2018', 'handle': '$294,006,820', 'revenue': '$8,433,994', 'hold': '2.9%', 'taxes': '$926,804'}
{'month': 'August 2018', 'handle': '$359,275,692', 'revenue': '$23,625,881', 'hold': '6.6%', 'taxes': '$2,524,419'}
{'month': 'September 2018', 'handle': '$817,681,310', 'revenue': '$95,358,434', 'hold': '11.7%', 'taxes': '$12,245,073'}
{'month': 'October 2018', 'handle': '$854,014,077', 'revenue': '$45,832,391', 'hold': '5.4%', 'taxes': '$5,217,752'}
{'month': 'November 2018', 'handle': '$996,071,884', 'revenue': '$54,349,323', 'hold': '5.5%', 'taxes': '$6,272,898'}
{'month': 'December 2018', 'handle': '$990,933,059', 'revenue': '$82,266,923', 'hold': '8.3%', 'taxes': '$11,262,831'}
{'month': 'January 2019', 'handle': '$1,002,092,655', 'revenue': '$43,052,327', 'hold': '4.3%', 'taxes': '$6,529,559'}
{'month': 'February 2019', 'handle': '$881,756,490', 'revenue': '

In [35]:
month_rev_data_df = pd.DataFrame(month_data_list)
month_rev_data_df.head(5)

,month,handle,revenue,hold,taxes
0,June 2018,"$309,695,659","$24,630,915",8.0%,"$2,221,650"
1,July 2018,"$294,006,820","$8,433,994",2.9%,"$926,804"
2,August 2018,"$359,275,692","$23,625,881",6.6%,"$2,524,419"
3,September 2018,"$817,681,310","$95,358,434",11.7%,"$12,245,073"
4,October 2018,"$854,014,077","$45,832,391",5.4%,"$5,217,752"


In [36]:
#create a dataframe from the market_data

#remove $ signs from the data and convert to float

#remove % signs from the data, convert to float, and divide by 100 

In [37]:
month_rev_data_df.to_csv('Resources/month_rev_data_df.csv', index=False)

3. Major Professional sports teams of the US and canada - Data did not scrape correctly due to table set up in Wikipedia - single cell overlap between the states etc--> pulled data by copying from excel --> if anyone wants to try you are more than welcome to dig in --> current file major_teams_usa.csv

5. build a large data scrape loop to get each states data in a dataframe --> see if possible  

4. Add geopify API to state list --> Map Leaflet Set up --> https://www.geoapify.com/boundaries-api

5. Census population data was downloaded directly from the census tabular menu - no need to do an API. - https://data.census.gov/table - https://data.census.gov/table?q=B05003&g=010XX00US$0400000&tid=ACSDT1Y2021.B05003 - 
B05003SEX BY AGE BY NATIVITY AND CITIZENSHIP STATUS

6. Summary by online vs in-person betting by state done via download - https://www.gamingtoday.com/states/ 